In [33]:
import csv,sys
import math,random
import subprocess
import numpy as np
import pandas as pd
from math import sqrt

In [34]:
class Point:
    
    def __init__(self,coords,label):
        self.coords = coords
        self.label = label
        self.n = len(coords)
    
    def __repr__(self):
        return str(self.coords)

In [35]:
class cluster_node:
    
    def __init__(self,points,left = None,right = None,distance = 0.0,Id = None,count =1):
        self.points = points
        self.left=left
        self.right=right
        self.distance = distance
        self.Id = Id
        self.count = count
    
    def __repr(self):
        return str(self.points)

In [36]:
def getDistance(a, b):

#Euclidean distance between two n-dimensional points.
#Note: This can be very slow and does not scale well
    #if a.n != b.n:
    #    raise Exception("ILLEGAL: non comparable points")
    #print type(a)
    ret = reduce(lambda x,y: x + pow((a[y]-b[y]), 2),range(len(a)),0.0)
    return math.sqrt(ret)
    #return sqrt(sum((a-b)**2))

In [37]:
#calculate minimum distance between two clusters
def get_min_distance(a,b,points,cutoff):
    cluster_id1 = get_cluster_elements(a)
    cluster_id2 = get_cluster_elements(b)
    min_distance = 1e+10
    for i in range(len(cluster_id1)):
        for j in range(len(cluster_id2)):
            distance = getDistance(points[i],points[j])
            if distance < min_distance:
                min_distance = distance
    return min_distance

In [38]:
def agglomerative_clustering_mean(points,k):
    distances={}
    currentclustid=-1
    cluster = [cluster_node(points[i],Id=i) for i in range(len(points))]
    #print len(cluster)
    while len(cluster) > k:
        lowestpair = (0,1)
        closest = getDistance(cluster[0].points,cluster[1].points)
        for i in range(len(cluster)):
            for j in range(i+1,len(cluster)):
                # distances is the cache of distance calculations
                if (cluster[i].Id,cluster[j].Id) not in distances: 
                    #print type(cluster[i].points.coords)
                    distances[(cluster[i].Id,cluster[j].Id)] = getDistance(cluster[i].points,cluster[j].points)

                d = distances[(cluster[i].Id,cluster[j].Id)]

                if d < closest:
                    closest=d
                    lowestpair=(i,j)

        # calculate the average of the two clusters
        mergeclust = [(cluster[lowestpair[0]].points[i]+cluster[lowestpair[1]].points[i])/2.0 
            for i in range(len((cluster[0].points)))]

        # create the new cluster
        newcluster = cluster_node(mergeclust,left = cluster[lowestpair[0]],
                             right=cluster[lowestpair[1]],
                             distance = closest,Id=currentclustid)

        # cluster ids that weren't in the original set are negative
        currentclustid-=1
        del cluster[lowestpair[1]]
        del cluster[lowestpair[0]]
        cluster.append(newcluster)
    return cluster

In [39]:
def agglomerative_clustering_single_link(points,k,cutoff):
    distances={}
    currentclustid=-1
    cluster = [cluster_node(points[i],Id=i) for i in range(len(points))]
    #print len(cluster)
    while len(cluster) > k:
        lowestpair = (0,1)
        closest = getDistance(cluster[0].points,cluster[1].points)
        min_distance = 1e+10
        for i in range(len(cluster)):
            for j in range(i+1,len(cluster)):
                if get_min_distance(cluster[i],cluster[j],points,cutoff) < min_distance:
                    min_distance = get_min_distance(cluster[i],cluster[j],points,cutoff)
                    lowestpair = (i,j)
        if min_distance < cutoff:
            mergeclust = [(cluster[lowestpair[0]].points[i]+cluster[lowestpair[1]].points[i])/2.0 
            for i in range(len((cluster[0].points)))]

        # create the new cluster
        newcluster = cluster_node(mergeclust,left = cluster[lowestpair[0]],
                             right=cluster[lowestpair[1]],
                             distance = closest,Id=currentclustid)

        # cluster ids that weren't in the original set are negative
        currentclustid-=1
        del cluster[lowestpair[1]]
        del cluster[lowestpair[0]]
        cluster.append(newcluster)
    return cluster

In [40]:
def main():
    data = pd.read_csv("arcene_train.data",delimiter = ' ')
    data = np.array(data)
    #np.any(np.isnan(data))
    data = list(data)
    #print data[0]
    temp = []
    for row in data:
        temp.append([int(elements) for elements in row[:10000]])
    data = temp
    labels = pd.read_csv("arcene_train.labels", delimiter = ' ')
    labels = np.array(labels)
    labels = list(labels)
    temp = []
    for row in labels:
        temp.append(int(row))
    labels = temp
    #points = []
    #for i in range(len(data)):
        #points.append(Point(data[i],labels[i]))
    num_clusters = 2
    opt_cutoff = 0.5
    flag = 2
    if flag == 1: 
        clusters = agglomerative_clustering_mean(data,num_clusters)
    else:
        clusters = agglomerative_clustering_single_link(data,num_clusters,opt_cutoff)
    print len(clusters)
    count_class = dict()
    accuracy = 0.0
    for i in range(num_clusters):
        cluster_id = get_cluster_elements(clusters[i])
        for j in range(len(cluster_id)):
            if labels[cluster_id[j]] in count_class.keys():
                count_class[labels[cluster_id[j]]]+=1
            else:
                count_class[labels[cluster_id[j]]]=1
        max_count = 0
        for key in count_class:
            if count_class[key] > max_count:
                max_count = count_class[key]
        accuracy += max_count
        count_class.clear()
    accuracy = accuracy/len(data)
    print accuracy

In [41]:
def get_cluster_elements(clust):
    # return ids for elements in a cluster sub-tree
    if clust.Id>0:
        # positive id means that this is a leaf
        return [clust.Id]
    else:
        # check the right and left branches
        cl = []
        cr = []
        if clust.left!=None: 
            cl = get_cluster_elements(clust.left)
        if clust.right!=None: 
            cr = get_cluster_elements(clust.right)
        return cl+cr

In [42]:
main()

2
0.555555555556
